In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import plotly.express as px

from datetime import datetime
from scipy import stats

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [7]:
#Pick Season
season = "2021-22"

#Import Files
all_players = pd.read_csv (r'./data/' + season + '/Merged_data/all_players_merged.csv', parse_dates= ["news_added"])
#all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv')
#player_gameweek_history = pd.read_csv(r'./data/2019-20/player_past_history/player_gameweek_history_raw.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])



In [8]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,279,2210549,18,NaN,1,NaN,28.0,False,0,False,2022-03-06 14:00:00,Gameweek 28,False,2,1,18,1
1,281,2210551,1,NaN,9,NaN,29.0,False,0,False,2022-03-13 16:30:00,Gameweek 29,True,3,1,9,1
2,261,2210531,1,NaN,11,NaN,29.0,False,0,False,2022-03-16 20:15:00,Gameweek 29,True,4,1,11,1
3,291,2210561,2,NaN,1,NaN,30.0,False,0,False,2022-03-19 12:30:00,Gameweek 30,False,3,1,2,1
4,304,2210574,7,NaN,1,NaN,31.0,False,0,False,2022-04-04 19:00:00,Gameweek 31,False,2,1,7,1


In [9]:
def player_future(player_id):
    data = player_future_fixture[player_future_fixture.player_id == player_id]

    player = all_players[all_players.player_id == player_id]
    team = data.team_id.iloc[0]
    goals_scored = player.iloc[0].goals_scored
    player_name = player.iloc[0]['web_name']

    fig = px.line(
        data, 
        x = 'event_name', 
        y = 'difficulty', 
        title = player_name,
        hover_data=['opponent_team_id'])

    fig.show()

In [10]:
player_future(150)

In [11]:
    opponent_strength = player_future_fixture['difficulty']
    opponent_strength
    #opponent_strength = player_analysis.iloc[:,2]

0       2
1       3
2       4
3       3
4       2
       ..
9367    2
9368    2
9369    2
9370    3
9371    3
Name: difficulty, Length: 9372, dtype: int64

In [12]:
def get_SMA(player_id , count):
    data = player_future_fixture[player_future_fixture.player_id == player_id]
    opponent_strength = data['difficulty']
    # List to store moving average results
    SMA2 = list(range(0, len(opponent_strength) - count))

    # Compute moving average
    for ii in range(len(SMA2)):
        # Previous gw index
        index = range(ii, ii + count)
        # Get data for previous gw
        prev_gw2 = opponent_strength.iloc[index]

        # Sum previous gw
        summation2 = np.sum(prev_gw2)

        # Get average
        avg2 = summation2/count
        # Save results to list
        SMA2[ii] = avg2
         
    # Define column label
    label2 = f"Opponent strength SMA"

    
    # Get corresponding gw for moving_avg
    gameweeks2 = opponent_strength.index[:-count]
    gameweek = data.event_name[:-count]
    
    # Convert list into Pandas Series
    SMA2 = pd.Series(SMA2, name = label2, index = gameweek)
    
    
    return SMA2

In [13]:
def player_future_avg(player_id , no_games):
    data = get_SMA(player_id , no_games)

    player = all_players[all_players.player_id == player_id]

    player_name = player.iloc[0]['web_name']
    title = "Average difficulty next " + str(no_games) + " games for " + str(player_name)

    fig = px.line(
        data, 
        title = title)

    fig.show()

In [14]:
player_future_avg(100 , 8)

In [15]:
result = get_SMA(150,4)
result

event_name
Gameweek 27    3.25
Gameweek 28    3.50
Gameweek 29    3.25
Gameweek 30    2.50
Gameweek 31    2.50
Gameweek 32    2.25
Gameweek 33    2.00
Gameweek 34    2.25
Name: Opponent strength SMA, dtype: float64

## FANTASY PLAYER VALUE

In [16]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost',
       'photo', 'PPG', 'second_name', 'selected_by_percent', 'status',
       'team_id', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'total_minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'thre

In [17]:
top_players = all_players[['web_name' , 'now_cost', 'status' , 'player_type_id', 'total_points' , 'total_minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['total_minutes']>100][all_players['status'] != "i"].sort_values(by='VAPM' , ascending = False).head(60)
top_players.head(20)

C:\Users\daniel.satre\AppData\Local\Temp\ipykernel_36376\523373215.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
599,Stones,52,a,2,45,591,6.852792,5.6,1.076923,0.692308
257,Torres,67,u,3,26,303,7.722772,6.5,0.970149,0.671642
535,Sarr,49,a,2,21,343,5.510204,5.2,1.061224,0.653061
526,Roberts,45,a,2,34,555,5.513514,4.9,1.088889,0.644444
557,Branthwaite,40,a,2,9,145,5.586207,4.5,1.125000,0.625000
475,Tierney,51,a,2,92,1546,5.355757,5.1,1.000000,0.607843
26,Arrizabalaga,47,a,1,19,360,4.750000,4.8,1.021277,0.595745
587,Alexander-Arnold,85,a,2,162,2043,7.136564,7.0,0.823529,0.588235
3,Ramsdale,51,a,1,105,1880,5.026596,5.0,0.980392,0.588235
586,Robertson,72,a,2,118,1664,6.382212,6.2,0.861111,0.583333


In [18]:
top_players[top_players['player_type_id'] == 1].head(3)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
26,Arrizabalaga,47,a,1,19,360,4.750000,4.8,1.021277,0.595745
3,Ramsdale,51,a,1,105,1880,5.026596,5.0,0.980392,0.588235
80,Sá,52,a,1,119,2240,4.781250,4.8,0.923077,0.538462


In [19]:
top_players[top_players['player_type_id'] == 2].head(5)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
599,Stones,52,a,2,45,591,6.852792,5.6,1.076923,0.692308
535,Sarr,49,a,2,21,343,5.510204,5.2,1.061224,0.653061
526,Roberts,45,a,2,34,555,5.513514,4.9,1.088889,0.644444
557,Branthwaite,40,a,2,9,145,5.586207,4.5,1.125000,0.625000
475,Tierney,51,a,2,92,1546,5.355757,5.1,1.000000,0.607843


In [20]:
top_players[top_players['player_type_id'] == 3].head(5)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
257,Torres,67,u,3,26,303,7.722772,6.5,0.970149,0.671642
170,Gallagher,62,a,3,117,1929,5.458787,5.3,0.854839,0.532258
353,Bowen,71,a,3,149,2218,6.045987,5.7,0.802817,0.521127
114,Coutinho,73,a,3,29,352,7.414773,5.8,0.794521,0.520548
239,Salah,131,a,3,209,2083,9.030245,8.7,0.664122,0.511450


In [21]:
top_players[top_players['player_type_id'] == 4].head(3)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
461,Dennis,61,a,4,107,1818,5.297030,4.7,0.770492,0.442623
403,Weghorst,65,a,4,23,429,4.825175,4.6,0.707692,0.400000


In [22]:
price_gk = top_players['now_cost'][top_players['player_type_id'] == 1].head(2).sum()
price_def = top_players['now_cost'][top_players['player_type_id'] == 2].head(5).sum()
price_mid = top_players['now_cost'][top_players['player_type_id'] == 3].head(5).sum()
price_fwd = top_players['now_cost'][top_players['player_type_id'] == 4].head(3).sum()


team_cost = price_gk + price_def + price_mid + price_fwd
print("Price for the best team is " + str(np.divide(team_cost,10)))

Price for the best team is 86.5


In [23]:
def player_value():

    fig = px.scatter(
        top_players.head(30), 
        x = 'now_cost', 
        y = 'VAPM',
        #size = 'minutes',
        color = 'total_points',
        text = 'web_name',
        hover_data=['PPG'])

    fig.show()

In [24]:
player_value()